In [2]:
# Dashboard 8013

import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load dữ liệu
df = pd.read_csv("df_final.csv")
df_unique = df.drop_duplicates(subset='orderid').reset_index(drop=True)
df_unique_customers = df.drop_duplicates(subset='customerid').reset_index(drop=True)

# Khởi tạo app
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True

# Filter options
time_options = sorted(df_unique['Time of day'].dropna().unique())
age_groups = sorted(df_unique['Age Group'].dropna().unique())
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

app.layout = html.Div([
    html.H1("📊 Dashboard Phân Khúc Khách Hàng ủa Rạp Phim", style={'textAlign': 'center', 'color': '#003049'}),

    html.Div([
        html.Div([
            html.Label("Thời gian trong ngày:"),
            dcc.Dropdown(
                options=[{'label': t, 'value': t} for t in time_options],
                id='time-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.Label("Nhóm tuổi:"),
            dcc.Dropdown(
                options=[{'label': a, 'value': a} for a in age_groups],
                id='age-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.Label("Thứ trong tuần:"),
            dcc.Dropdown(
                options=[{'label': w, 'value': w} for w in weekdays],
                id='weekday-filter', value=None, placeholder="Tất cả"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'})
    ]),

    html.Div([
        html.Div(dcc.Graph(id='district-bar-chart'), style={'width': '33%', 'display': 'inline-block'}),
        html.Div(dcc.Graph(id='group-pie-chart'), style={'width': '33%', 'display': 'inline-block'}),
        html.Div(dcc.Graph(id='gender-pie-chart'), style={'width': '33%', 'display': 'inline-block'})
    ]),

    html.Div([
        html.Div(dcc.Graph(id='job-distribution-chart'), style={'width': '33%', 'display': 'inline-block'}),
        html.Div(dcc.Graph(id='age-group-chart'), style={'width': '33%', 'display': 'inline-block'}),
        html.Div(dcc.Graph(id='industry-distribution-chart'), style={'width': '33%', 'display': 'inline-block'})
    ])
])

# Callback: Quận/Huyện
@app.callback(
    Output('district-bar-chart', 'figure'),
    Input('time-filter', 'value')
)
def update_district_chart(_):
    counts = df_unique_customers['Quận/Huyện'].value_counts().reset_index()
    counts.columns = ['Quận/Huyện', 'Số lượng khách hàng']
    fig = px.bar(counts, x='Quận/Huyện', y='Số lượng khách hàng',
                 title='Số lượng khách hàng đặt vé theo Quận/Huyện ở Đà Nẵng',
                 color='Quận/Huyện',
                 color_discrete_sequence=px.colors.sequential.Blues)
    fig.update_traces(text=counts['Số lượng khách hàng'], textposition='outside')
    return fig

# Callback: Nhóm đi cùng
@app.callback(
    Output('group-pie-chart', 'figure'),
    Input('time-filter', 'value')
)
def update_group_pie(_):
    # Tính số người trong mỗi đơn hàng
    order_size = df.groupby('orderid')['customerid'].count()

    # Phân loại nhóm người
    def classify_group(n):
        if n == 1:
            return "Đi 1 mình"
        elif n == 2:
            return "Nhóm 2 người"
        elif n == 3:
            return "Nhóm 3 người"
        elif n == 4:
            return "Nhóm 4 người"
        else:
            return "Nhóm > 4 người"

    group_df = order_size.reset_index(name='group_size')
    group_df['group_label'] = group_df['group_size'].apply(classify_group)

    group_ratio = group_df['group_label'].value_counts(normalize=True).reset_index()
    group_ratio.columns = ['Nhóm', 'Tỷ lệ']
    group_ratio['Tỷ lệ'] = group_ratio['Tỷ lệ'] * 100

    fig = px.pie(group_ratio, names='Nhóm', values='Tỷ lệ',
                 title='Tỷ lệ khách đi theo nhóm',
                 color_discrete_sequence=['#102E50', '#6F826A', '#FDF0D5', '#97866A', '#669BBC'])
    return fig

# Callback: Giới tính
@app.callback(
    Output('gender-pie-chart', 'figure'),
    Input('time-filter', 'value')
)
def update_gender_pie(_):
    counts = df_unique_customers['Gender'].value_counts().reset_index()
    counts.columns = ['Giới tính', 'Số lượng']
    fig = px.pie(counts, names='Giới tính', values='Số lượng',
                 title='Tỷ lệ khách hàng theo giới tính (Chart 2)',
                 color_discrete_sequence=['#102E50', '#A6D6D6'])
    return fig

# Callback: Công việc
@app.callback(
    Output('job-distribution-chart', 'figure'),
    Input('time-filter', 'value')
)
def update_job_chart(_):
    job_counts = df_unique_customers['job'].value_counts().reset_index()
    job_counts.columns = ['Công việc', 'Số lượng']
    fig = px.bar(job_counts, x='Công việc', y='Số lượng',
                 title='Phân bố người đặt theo công việc',
                 labels={'Công việc': 'Công việc', 'Số lượng': 'Số lượng'},
                 color='Công việc',
                 color_discrete_sequence=px.colors.sequential.Cividis)
    return fig

# Callback: Nhóm tuổi
@app.callback(
    Output('age-group-chart', 'figure'),
    Input('time-filter', 'value')
)
def update_age_group_chart(_):
    age_counts = df_unique_customers['Age Group'].value_counts().reset_index()
    age_counts.columns = ['Nhóm tuổi', 'Số lượng']
    fig = px.bar(age_counts, x='Nhóm tuổi', y='Số lượng',
                 title='Phân bố người đặt theo nhóm tuổi',
                 color='Nhóm tuổi',
                 color_discrete_sequence=px.colors.sequential.Cividis)
    return fig

# Callback: Lĩnh vực
@app.callback(
    Output('industry-distribution-chart', 'figure'),
    Input('time-filter', 'value')
)
def update_industry_chart(_):
    industry_counts = df_unique_customers['industry'].value_counts().reset_index()
    industry_counts.columns = ['Lĩnh vực', 'Số lượng']
    fig = px.bar(industry_counts, x='Lĩnh vực', y='Số lượng',
                 title='Phân bố người đặt theo lĩnh vực',
                 color='Lĩnh vực',
                 color_discrete_sequence=px.colors.sequential.Cividis)
    return fig

if __name__ == '__main__':
    #app.run_server(debug=True)

    app.run(debug=True, port=8013)
